### 使用ChatGLM实现基于知识库的问答

大语言模型的一个典型场景就是在问答场景，使用更加人性化的方式，回答提出的问题。但是，目前的大语言模型，都是基于一定的数据集进行训练，模型本身所能回答的问题，也都是基于训练时所使用的数据集。当你问一些开放性的问题时，模型可以根据自己的推理和发散的能力，给出较为合理的答案，但是当你想问一些客观事实性的问题，如果这个问题的事实发生的时间在模型训练之后，或者模型训练没有使用相关的数据，那么模型就无法给出答案，甚至是编一个看似合理，实则错误的答案。


所以，我们可以先使用我们的知识库，从知识库中搜索出相关的内容，再调用语言模型的问答接口，以搜索出来的结果作为上下文，再加上问题，让语言模型从上下文中抽取出答案，进行回答。其大致流程如下：

![](imgs/knowledge-based-chatglm-qa.jpg)

### 使用医疗问答数据集生成向量知识库

为了构建行业知识库，我们使用[cMedQA2](https://github.com/zhangsheng93/cMedQA2)数据集，他是医学在线论坛的数据集，包含包含约10万个医学相关问题，及对应的约20万个回答。

在notebook左侧找到“Git Clone”的按钮，将该数据集（https://github.com/zhangsheng93/cMedQA2） 克隆到notebook里。

>在本Workshop中，为了方便，以及将处理好的CSV文件保存在当前目录中，甚至FAISS生成的索引文件也已经创建并保存好，不需要重新执行。

#### 预处理医疗问答数据
首先我们解压缩数据集，查看其内容和关系。数据集中，已经把训练数据、测试数据分开，在这里，我们只使用训练数据。

In [5]:
import shutil
import pandas as pd

shutil.unpack_archive("./cMedQA2/answer.zip", "./cMedQA2/")
shutil.unpack_archive("./cMedQA2/question.zip", "./cMedQA2/")
shutil.unpack_archive("./cMedQA2/train_candidates.zip", "./cMedQA2/")

In [5]:
import pandas as pd

df_qa = pd.read_csv("./cMedQA2/train_candidates.txt")
df_qa.head()

,question_id,pos_ans_id,neg_ans_id
0,24731702,11064,163246
1,24731702,11064,106174
2,24731702,11064,104592
3,24731702,11064,64458
4,24731702,11063,200971


In [6]:
df_questions = pd.read_csv("./cMedQA2/question.csv")
df_questions.head()

,question_id,content
0,65102009,头痛恶心肌肉痛关节痛颈部淋巴结疼痛怎么回事啊
1,44275784,我怀孕37周，肠子一直呼噜呼噜叫感觉像是在里面灌水，上厕所拉稀和喷水一样，一天上厕所5次，对...
2,42163349,男，67岁，2天前突然出现右小腿（类似抽筋症状），现出现右小腿消肿，有指压痕，无，请问可能是...
3,67935540,怀孕前两个月照了CT和X光，来两次月经后怀孕了，怀孕期间有轻微盆腔积水吃了两瓶杏香兔耳片。请...
4,33429289,阴囊湿冷阳痿早泻请问用点什么药


In [7]:
df_answers = pd.read_csv("./cMedQA2/answer.csv")
df_answers.head()

,ans_id,question_id,content
0,0,45619783,月经延迟十四天而且伴随恶心，头痛，乏力的现象，那么考虑怀孕的概率是非常大的，建议你去医院检查...
1,1,45619783,如果你的月经周期规律，有正常的性生活，未采取任何有效的避孕措施，此时的症状考虑有怀孕的可能。...
2,2,45619783,建议在性生活过后14天左右可以用怀孕试纸自我检测一下，一般怀孕试纸显示2条线的话是怀孕了的，...
3,3,26616465,头痛是临床上最为常见的临床症状之一，是人体对各种致痛因素所产生的主观感觉，属于疼痛的范畴。建...
4,4,26616465,头痛主要是由于头部的血管、神经、脑膜等对疼痛敏感的组织受到刺激引起的。由紧张、疲劳、饮酒等原...


df_qa的数据（也就是train_candidates.txt）是一个关系数据，包含了问题id、答案的id，及否定答案的id，这里我们只需要正确答案。而且，这个关系表中一个问题有多个答案和多个错误答案，所以，去掉错误答案那一列后去重。

In [8]:
# 只处理正确答案
df_qa = df_qa.drop('neg_ans_id', axis=1)
# 删除错误答案后，去掉重复的, 每个问题会有1个或多个答案
df_qa = df_qa.drop_duplicates()
df_qa.shape

(188490, 2)

最终有近19万条问答，根据数据集提供的信息，训练数据应该有10万条问题，188490个答案。我们去重后的结果显示有188490个答案。

In [9]:
df_questions = df_questions.rename(columns={"content": "question_content"})
df_answers = df_answers.rename(columns={"content": "answer_content"})

# join
df_qa_with_contents = df_qa.join(df_questions.set_index('question_id'), on = ['question_id'])
df_answers['pos_ans_id'] = df_answers["ans_id"]
df_qa_with_contents = df_qa_with_contents.join(df_answers.set_index('pos_ans_id'), on = 'pos_ans_id', lsuffix='', rsuffix='_other')
# 我们只需要问题和答案两列
df_qa_compact = df_qa_with_contents[['question_content', 'answer_content']]

In [10]:
df_qa_compact.head()

,question_content,answer_content
0,不是说做b超对宝宝不好吗？那怀孕检查是不？不是说做b超对宝宝不好吗？那怀孕检查是不是越少越好...,B超属于超声波经常检查是不好的而且也没有必要经常检查的一般怀孕两个月检查一下怀孕五个月检查一...
4,不是说做b超对宝宝不好吗？那怀孕检查是不？不是说做b超对宝宝不好吗？那怀孕检查是不是越少越好...,b超切实有一定的辐射，而且小孩比较的娇嫩，容易受辐射影响发育。宝宝尽量不要做b超，但是在胎儿...
50,腰酸并伴有小腹胀痛怎么回事啊，而且引导有少量的血？,可能是妇科炎症引起的建议放置达克宁栓口服三金片和龙胆泻肝丸如过不放心可以做一个支原体化验
52,腰酸并伴有小腹胀痛怎么回事啊，而且引导有少量的血？,就你的情况考虑盆腔炎等情况造成的，可到医院做B超检查来确定原因。
57,腰酸并伴有小腹胀痛怎么回事啊，而且引导有少量的血？,妇科出现腰酸和小腹胀痛的症状是和妇科病有关系的。很多原因都会导致腰酸。最常见的妇科疾病比如宫...


In [11]:
df_qa_compact.shape

(188490, 2)

#### 生成知识库的文本文件

下面，我们将问答内容写到一个文件里。

由于我们需要对每一对问答记录进行向量化，所以在保存的时候进行一定的格式化，方便之后分片。

In [12]:
df_qa_compact['answer_content'] = 'A:' + df_qa_compact['answer_content']
df_qa_compact['question_content'] = 'Q:' + df_qa_compact['question_content']

/tmp/ipykernel_32067/2516721465.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_qa_compact['answer_content'] = 'A:' + df_qa_compact['answer_content']
/tmp/ipykernel_32067/2516721465.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_qa_compact['question_content'] = 'Q:' + df_qa_compact['question_content']


In [13]:
df_qa_compact.head()

,question_content,answer_content
0,Q:不是说做b超对宝宝不好吗？那怀孕检查是不？不是说做b超对宝宝不好吗？那怀孕检查是不是越少...,A:B超属于超声波经常检查是不好的而且也没有必要经常检查的一般怀孕两个月检查一下怀孕五个月检...
4,Q:不是说做b超对宝宝不好吗？那怀孕检查是不？不是说做b超对宝宝不好吗？那怀孕检查是不是越少...,A:b超切实有一定的辐射，而且小孩比较的娇嫩，容易受辐射影响发育。宝宝尽量不要做b超，但是在...
50,Q:腰酸并伴有小腹胀痛怎么回事啊，而且引导有少量的血？,A:可能是妇科炎症引起的建议放置达克宁栓口服三金片和龙胆泻肝丸如过不放心可以做一个支原体化验
52,Q:腰酸并伴有小腹胀痛怎么回事啊，而且引导有少量的血？,A:就你的情况考虑盆腔炎等情况造成的，可到医院做B超检查来确定原因。
57,Q:腰酸并伴有小腹胀痛怎么回事啊，而且引导有少量的血？,A:妇科出现腰酸和小腹胀痛的症状是和妇科病有关系的。很多原因都会导致腰酸。最常见的妇科疾病比...


In [49]:
import os
path = "contents"
if not os.path.exists(path):
   os.makedirs(path)
df_qa_compact.to_csv('contents/med_qa.csv', index=False, header=False, sep='\n', quotechar=' ', lineterminator='\n\n')

打开terminal，检查文件内容如下：
```bash
sh-4.2$ head med_qa.csv 
Q:不是说做b超对宝宝不好吗？那怀孕检查是不？不是说做b超对宝宝不好吗？那怀孕检查是不是越少越好。无麻烦解答，谢谢。
A:B超属于超声波经常检查是不好的而且也没有必要经常检查的一般怀孕两个月检查一下怀孕五个月检查一下快出生时在检查就可以还有就是不舒服检查就可以的

Q:不是说做b超对宝宝不好吗？那怀孕检查是不？不是说做b超对宝宝不好吗？那怀孕检查是不是越少越好。无麻烦解答，谢谢。
A:b超切实有一定的辐射，而且小孩比较的娇嫩，容易受辐射影响发育。宝宝尽量不要做b超，但是在胎儿期有母体的保护，所以不要担心，有必要的话一定要做。

Q:腰酸并伴有小腹胀痛怎么回事啊，而且引导有少量的血？
A:可能是妇科炎症引起的建议放置达克宁栓口服三金片和龙胆泻肝丸如过不放心可以做一个支原体化验
```

#### LangChain AI应用开发框架
在开始之前，我们需要了解我们在本文中主要用到的框架 [LangChain](https://github.com/hwchase17/langchain).

顾名思义，LangChain就是基于 LLM 语言模型开发AI应用的框架，它通过链式调用的方式，将AI应用中的各个过程串联起来，实现一系列的功能。举例来说，OpenAI的ChatGPT默认不能上网，它所能回答的内容是基于它的训练数据，如果想要基于新的互联网上的某个新闻，让ChatGPT回答某些问题，就可以通过 LangChain 来实现。

在LangChain里，提供了各种 Chain，有能够请求API获得结果的Chain，也有某个LLM语言模型的Chain，也有调用Python代码的Chain。使用这些 Chain，我们就可以实现“上网搜索某些内容，再根据结果提取想要的答案”这样的场景，也可以“让ChatGPT帮我们生成一段Python代码，然后在本地执行该代码并返回结果”。

在 LangChain 里，以及实现了很多有意思的 Chain，比如说 “[Chat Over Documents with Chat History](https://python.langchain.com/en/latest/modules/chains/index_examples/chat_vector_db.html)”，利用该 Chain，我们可以给AI提供一本小说，让AI以该小说的设定和内容，来和我们实现多轮对话。

还有一个就是"[Retrieval Question/Answering](https://python.langchain.com/en/latest/modules/chains/index_examples/vector_db_qa.html)"，他就是用于从向量数据库中进行相似搜索，然后从相似内容中回答我们的问题。

安装几个依赖的库。
* langchain：即应用框架
* faiss-gpu：本地向量库

其中 faiss-gpu 是GPU版本的库，如果使用的notebook不是GPU实例，那就安装 faiss-cpu。但是cpu进行embedding也是会比gpu慢很多。

In [ ]:
!pip install langchain
!pip install faiss-gpu

下面是本workshop使用的 LangChain 的版本，需要注意这个框架发展的非常快，几天就会发布一个小版本。如果版本差别过大，可能需要更新代码。

In [23]:
from langchain import __version__
__version__

'0.0.176'

#### 向量化并保存
文本内容的向量数据库包含两个方面：
1. 向量化，即将文本生成向量
2. 向量化数据库，即对生成的向量进行保存，并提供相似搜索等功能


首先要对一段文本向量化，我们需要使用某个模型，在OpenAI中，使用 'text-embedding-ada-002' 模型生成文本的向量，OpenAI 也提供了API可以直接生成向量。

这里，我们使用 [GanymedeNil/text2vec-large-chinese](https://huggingface.co/GanymedeNil/text2vec-large-chinese)，它是一个隐层大小1024的模型，同时该作者也提供了768的模型。

>实际上，本问的思想也是来自于该模型作者的库[document.ai](https://github.com/GanymedeNil/document.ai)，即通过加载一个文档，进行向量化并进行相似搜索，然后通过 ChatGPT 提取答案信息。

向量数据库的选择也非常多，常用的有 ElasticSearch、Redis、Postgres vector database (PGVector）等，或者专门的向量数据，如开源的向量数据库Milvus。这里，由于我们的知识库的内容也就不到20万，完全可以使用基于内存的本地向量数据库，所以我们选择使用[FAISS](https://faiss.ai/)，一个由Facebook开源的库。

In [42]:
# 文件分片的时候需要设置大小
max_question_len = df_qa_compact.question_content.map(len).max()
max_answer_len = df_qa_compact.answer_content.map(len).max()

print("max_question_len:" + str(max_question_len))
print("max_answer_len:" + str(max_answer_len))

max_question_len:151
max_answer_len:251


LangChain已经提供了很多分片的工具类，但是因为我们想要每一对问答作为一个分片，所以自己进行分片并生成documents

In [5]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document

with open('./contents/med_qa.csv') as f:
    med_qa_file = f.read()

documents = []
for qa_text in med_qa_file.split('\n\n'):
    if qa_text:
        new_doc = Document(
            page_content=qa_text, metadata={}
        )
        documents.append(new_doc)

In [6]:
# documents的长度与问答对的长度一致。
len(documents)

188490

In [7]:
documents[0]

Document(page_content='Q:不是说做b超对宝宝不好吗？那怀孕检查是不？不是说做b超对宝宝不好吗？那怀孕检查是不是越少越好。无麻烦解答，谢谢。\nA:B超属于超声波经常检查是不好的而且也没有必要经常检查的一般怀孕两个月检查一下怀孕五个月检查一下快出生时在检查就可以还有就是不舒服检查就可以的', metadata={})

#### 部署 Embedding 模型
下面，我们从HuggingFace上部署Embedding模型 “GanymedeNil/text2vec-large-chinese”。

因为我们只是生成Embedding向量，生成之后就不再需要，所以我们在这个Notebook实例里面部署该模型。它会从HuggingFace上下载模型，并缓存到本地。

In [ ]:
# 安装 sentence_transformers
# !pip install sentence_transformers

In [3]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name='GanymedeNil/text2vec-large-chinese')

No sentence-transformers model found with name /home/ec2-user/.cache/torch/sentence_transformers/GanymedeNil_text2vec-large-chinese. Creating a new one with MEAN pooling.


In [8]:
# 尝试计算embedding
doc_embed = embeddings.embed_query(documents[0].page_content)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
# 查看生成的embedding向量的维度
len(doc_embed)

1024

下面就对我们所有的documents进行embedding向量化。

先看看对100个问答生成向量需要多久。

In [10]:
%%time
from langchain.vectorstores import FAISS
FAISS.from_documents(documents[0:100], embeddings)

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

CPU times: user 2min 6s, sys: 30.5 s, total: 2min 36s
Wall time: 2min 37s


上面的语句，使用G4dn的机型，结果大概需要3.6秒，但是在使用cpu的情况下，需要差不多160秒。

在这个demo中，有近19万条问答，用g4dn完成整个向量化需要花费近3个小时，所以，我们将生成后的结果保存成文件，这样就可以重用这个文件，而无需每次都重新生成。

由于这个index文件比较大，就没有传到代码库中，读者需要自己生成index使用，建议使用GPU的notebook，并且选其中一部分文档生成。避免时间太长

In [ ]:
%%time

from langchain.vectorstores import FAISS
vector_store = FAISS.from_documents(documents, embeddings)
vs_path = './med_qa_FAISS_index'
vector_store.save_local(vs_path)

#### 我们可以直接从之前创建好的FAISS索引文件导入

In [11]:
from langchain.vectorstores import FAISS
vector_store = FAISS.load_local("./med_qa_FAISS_index", embeddings)

执行相似搜索查看结果：

In [12]:
vector_store.similarity_search('眼睛干涩怎么办？', k=3)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[Document(page_content='Q:眼睛干涩是什么症状？该怎么办？\nA:干眼症，在长期使用电脑，喜欢看电视，看书，在空气环境较差的地方工作生活都可以导致此病发生。对于老年人来说，由于年龄增长睑板腺的分泌功能减退，泪液成分改变，更容易蒸发，出现干涩症状。临床表现是眼干涩，眼疲劳，异物感，刺痛流泪', metadata={}),
 Document(page_content='Q:最近眼睛感觉很干涩，想流泪。\nA:眼睛干涩也就是指医学上所说的"眼结膜干燥症"也称作干眼症，它不是单一的某一种疾病，而是由几种完全不同的病因所引起、以眼干少泪为特征的一组疾病。说到引起眼睛干涩的原因，要是往前了说肯定是指上火一类的原因，但现今社会引起眼睛干涩的原因大多数是用眼过度，而这当中数长期对着电脑工作的人为主。', metadata={}),
 Document(page_content='Q:我眼睛一直干涩，怕光，老流泪眼角痒疲劳，眼睛可难受了，我买的珍珠明目什么作用也没有起，请问是怎么回事？谢谢\nA:眼睛干涩一般和近视无关，主要还是可能为由于用眼过度或熬夜引起的。眼睛眨眼次数减少，泪液分泌不足会引起眼睛干涩，长期存在就会形成干眼症。最好是到医院眼科进行泪液分泌检查及裂隙灯角膜检查等明显，对应可使用人工泪液缓解眼睛干涩，同时需要注意眼睛的休息。如果存在角膜磨损，最好是配合使用贝复舒滴眼液促进修复。', metadata={})]

也可以同向量similarity的分数一起返回，我们可以根据返回的分数设定一个阈值，当相似性小于一定的值就不返回，这样可以避免查找出来不相关的结果。

但是，不同的工具得到的similarity的分数不太一样。例如这里使用的Faiss，计算的结果是一个距离值，值越大代表越不相似。

In [12]:
vector_store.similarity_search_with_score('眼睛干涩怎么办？', k=3)

[(Document(page_content='Q:眼睛干涩是什么症状？该怎么办？\nA:干眼症，在长期使用电脑，喜欢看电视，看书，在空气环境较差的地方工作生活都可以导致此病发生。对于老年人来说，由于年龄增长睑板腺的分泌功能减退，泪液成分改变，更容易蒸发，出现干涩症状。临床表现是眼干涩，眼疲劳，异物感，刺痛流泪', metadata={}),
  327.9027),
 (Document(page_content='Q:最近眼睛感觉很干涩，想流泪。\nA:眼睛干涩也就是指医学上所说的"眼结膜干燥症"也称作干眼症，它不是单一的某一种疾病，而是由几种完全不同的病因所引起、以眼干少泪为特征的一组疾病。说到引起眼睛干涩的原因，要是往前了说肯定是指上火一类的原因，但现今社会引起眼睛干涩的原因大多数是用眼过度，而这当中数长期对着电脑工作的人为主。', metadata={}),
  342.84564),
 (Document(page_content='Q:我眼睛一直干涩，怕光，老流泪眼角痒疲劳，眼睛可难受了，我买的珍珠明目什么作用也没有起，请问是怎么回事？谢谢\nA:眼睛干涩一般和近视无关，主要还是可能为由于用眼过度或熬夜引起的。眼睛眨眼次数减少，泪液分泌不足会引起眼睛干涩，长期存在就会形成干眼症。最好是到医院眼科进行泪液分泌检查及裂隙灯角膜检查等明显，对应可使用人工泪液缓解眼睛干涩，同时需要注意眼睛的休息。如果存在角膜磨损，最好是配合使用贝复舒滴眼液促进修复。', metadata={}),
  350.25348)]

#### 构建知识库的要点
1. 文档内容提取及分片，特别是有图文、表格等复杂格式的文档。
2. 进行相似搜索后根据相似值、或距离值进行阈值判断。

### 使用ChatGLM基于知识库进行问答
下面，我们使用 LangChain 创建一个基于知识库的 Chain，来实现问答。

#### 从 Sagemaker Endpoint 创建model
之前我们创建了ChatGLM的endpoint，我们使用 LangChain 提供的封装类*SagemakerEndpoint*，将该endpoint生成model对象。

In [16]:
import logging
logger = logging.getLogger()
logging.basicConfig(level=logging.INFO)
logger.info("test")

INFO:root:test


In [18]:
from typing import Dict
from langchain.prompts import PromptTemplate
from langchain import SagemakerEndpoint
from langchain.llms.sagemaker_endpoint import LLMContentHandler
from langchain.chains.question_answering import load_qa_chain
import json

from langchain.chains import RetrievalQA

import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

sagemaker_endpoint_name = "mt-chatglm-6b-entpoint"

class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs: Dict) -> bytes:
        input = {"ask": prompt, **model_kwargs}
        logger.info("prompt: %s", prompt)
        logger.info("model_kwargs: %s", model_kwargs)
        input_str = json.dumps(input)
        return input_str.encode('utf-8')
    
    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        logger.info("response_json: %s", response_json)
        return response_json["answer"]

content_handler = ContentHandler()

chatglm_model = SagemakerEndpoint(
    endpoint_name=sagemaker_endpoint_name, 
    region_name="us-east-1", 
    model_kwargs={"temperature": 0.1},
    content_handler=content_handler
)

prompt_template = """使用下面的已知内容，简洁、准确的回答最后的问题，并使用中文。如果你不知道答案，就直接返回'根据已知信息无法回答该问题。'，不要编造答案。

已知内容:
{context}

问题: {question}
答案:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}

chain = RetrievalQA.from_chain_type(
    llm=chatglm_model, chain_type="stuff", retriever=vector_store.as_retriever(),
    chain_type_kwargs=chain_type_kwargs
)


INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


下面，使用 LangChain 提供的 *RetrievalQA* 创建一个QA Chain，用于进行问答。

就可以使用这个 Chain 进行问答。

In [19]:
chain({"query": "尿酸偏高怎么办？"})

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:prompt: 使用下面的已知内容，简洁、准确的回答最后的问题，并使用中文。如果你不知道答案，就直接返回'根据已知信息无法回答该问题。'，不要编造答案。

已知内容:
Q:降尿酸的药有哪些是比较好的？检查发现尿酸高了，还没发作，现在在吃舒一肽，提前调理防止痛风。这种情况平时要注意些什么呢？降尿酸的药有哪些是比较好的？
A:血尿酸正常值为：男149-416umol/L。女89-357umol/L。尿酸过高容易导致痛风的发生和肾功能损伤。对于降低尿酸主要通过减少摄入（避免豆类、动物内脏、海鲜、啤酒等）。减少尿酸生成（服用别嘌醇药物）。和增加排泄（多喝水，服用小苏打碱化尿液，促进其排泄）等方式。

Q:降尿酸的药有哪些是比较好的？检查发现尿酸高了，还没发作，现在在吃舒一肽，提前调理防止痛风。这种情况平时要注意些什么呢？降尿酸的药有哪些是比较好的？
A:病例分析:痛风、高尿酸血症分型诊断很重要

Q:我是一名尿急的患者，虽然这个病不是什么大病，但是得了以后，每天还是挺难熬的，请问尿急是什么原因啊？为什么我会得这病？
A:最常见为炎症性刺激，如肾盂肾炎、肾结石合并感染、肾结核、膀胱炎、尿道炎、前列腺炎、阴道炎。在急性炎症和活动性泌尿系结核时最为明显。再者为非炎症性刺激如膀胱结石、尿道结石、输尿管下l/3段结石等，以及膀胱肿瘤、尿道肿瘤、前列腺肿瘤等、膀胱或尿道内异物、膀胱屡和妊娠压迫等刺激。控制饮食结构避免酸性物质摄入过量加剧酸性体质饮食的酸碱平衡对于尿频的预防是非常重要的一个环节饮食方面要多吃富含植物有机活性碱的食品少吃肉类多吃蔬菜恰玛古富含植物有机活性碱能迅速排除体内酸毒使达到酸碱平衡保持一个健康的

Q:尿酸高的原因去医院查，我患上痛风，尿酸很高，痛风怎么办？痛风怎么办？
A:尿酸偏低：尿酸偏低可能是进食蛋白质食物较少一般对身体没有影响。不知道你的尿酸偏低程度，因为肾性低尿酸血症，家族性低尿酸血症，多发性骨髓肿瘤，酒精性肝障碍，原发性胆汁性肝硬化，糖尿病性肾症初期，黄嘌呤氧化酵素过剩症都有可能导致尿酸低。尿酸低是没坏处的但是可能说明蛋白质摄入得少了，多补充蛋白质看看能不能有所调养。实在不行建议到医院做个全面检查。

问题: 尿酸偏高怎么办？
答案:
INFO:root:model_kwargs: {'temperature': 0.1}
INFO:r

{'query': '尿酸偏高怎么办？',
 'result': '根据已知内容，回答最后的问题：\n\n如果尿酸偏高，建议通过以下方式之一来降低尿酸水平：\n\n1. 减少摄入：避免食用高嘌呤食物，如豆类、动物内脏、海鲜、啤酒等。\n2. 减少尿酸生成：服用别嘌醇药物。\n3. 增加排泄：多喝水，服用小苏打碱化尿液，促进其排泄。\n\n降尿酸的药物有很多，但具体选择应根据医生的建议。'}

In [152]:
chain({"query": "眼睛干涩怎么办？"})

{'query': '眼睛干涩怎么办？',
 'result': '根据已知信息，可以得出结论：眼睛干涩可能是由多种因素引起的，包括长时间使用电脑、过度眨眼、熬夜、干燥的空气等。珍珠明目可能无法完全解决眼睛干涩的问题，建议到眼科进行泪液分泌检查和裂隙灯角膜检查，根据医生的建议使用人工泪液和适当的治疗方法。如果存在角膜磨损，还需要配合使用贝复舒滴眼液促进修复。'}

In [21]:
# 问一个知识库中应该没有的问题
chain({"query": "为什么爸妈结婚没叫我参加婚礼？"})

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:prompt: 使用下面的已知内容，简洁、准确的回答最后的问题，并使用中文。如果你不知道答案，就直接返回'根据已知信息无法回答该问题。'，不要编造答案。

已知内容:
Q:结婚2年了，始终没有小孩，老婆检查没事，说我是精子成活率低，该怎么办我也要咨询
A:从你提供的临床资料来看，结婚2年了始终没有小孩精子成活率低考虑属于男性不育症的可能性较大，临床上男性不育较少见。建议请当地的中医师望闻问切、辨证施治，采用如五子衍宗丸加减化裁煎汤治疗，必要的话要考虑调换工种，希望能够帮助到你。

Q:大庆戒酒中心我男朋友朋友的伯父是精神病，和她结婚，我们的小孩会不会有可能得精神病。我们的男朋友以后会不会有精神病？
A:大多数的精神病人是出生缺陷和先天遗传、个性特征及体质因素、器质因素是其中一少部分。如果他的爷爷奶奶没有精神问题，他的父母也没有精神问题，你们的小孩不会有问题的。

Q:和智力障碍的人结婚会影响后代吗
A:这个要看对方智利障碍是先天性还是后天性因素所致。如果对方的长辈、亲人中也有智利障碍的患者，则说明有家族史，可能会有一定的几率遗传。如果是后天性因素，比如说分娩时缺氧导致的智力障碍，或者头部外伤后遗症等，这类情况智力障碍一般不会遗传。建议你可以再详细了解下对方智利障碍的原因。

Q:怀孕74天，医生说要建小卡，可我还没有领结婚证，我想问下南京市妇幼保健院没有结婚证能建小卡做产检吗
A:这个是可以进行建卡的，不需要结婚证的。主要填写你个人信息而已。当然，具体要根据医院要求进行。另外建议尽快领证，避免孩子出生时候办不了出生证明。

问题: 为什么爸妈结婚没叫我参加婚礼？
答案:
INFO:root:model_kwargs: {'temperature': 0.1}
INFO:root:response_json: {'answer': '根据已知内容，无法回答该问题。因为父母结婚时，你没有出生，所以无法参加婚礼。'}


{'query': '为什么爸妈结婚没叫我参加婚礼？',
 'result': '根据已知内容，无法回答该问题。因为父母结婚时，你没有出生，所以无法参加婚礼。'}

In [186]:
chain({"query": "愤怒会得心脏病吗"})
# chain.run("愤怒会得心脏病吗")

{'query': '愤怒会得心脏病吗',
 'result': '根据已知信息无法回答该问题，因为愤怒本身不会直接导致心脏病。然而，长期的愤怒和情绪不稳定可能会对身体产生负面影响，导致心血管疾病和其他健康问题。因此，保持良好的心理健康和情绪管理非常重要，包括通过适当的沟通、放松练习和体育锻炼等方式来缓解情绪不稳定的症状。'}

可以看到，当知识库中找到的结果几乎没有相关性时，应该从上下文中找不到结果。这时，使用 ChatGLM 的结果中，会先显示 “根据已知信息无法回答该问题”，但是紧接着就会使用模型中已有的知识进行回答。

In [28]:
chatglm_model._call(prompt="愤怒会得心脏病吗", stop=['，'])

INFO:root:prompt: 愤怒会得心脏病吗
INFO:root:model_kwargs: {'temperature': 0.1}
INFO:root:input: {'ask': '愤怒会得心脏病吗', 'temperature': 0.1}
INFO:root:response_json: {'answer': '愤怒可能会对心脏健康产生负面影响，但并非一定会导致心脏病。\n\n愤怒可能会导致血压升高，增加心脏病的风险。长期的高血压升高会增加患心脏病和中风的风险。\n\n愤怒可能会使心率加快，导致心律不齐，增加心脏病的风险。\n\n愤怒可能会影响心理状态，降低心理健康的风险。长期的愤怒和抑郁可能会对心脏健康产生负面影响。\n\n因此，虽然愤怒可能会对心脏健康产生负面影响，但保持适当的情绪状态和心理健康对于心脏健康非常重要。'}


'愤怒可能会对心脏健康产生负面影响'

如果我们问一个知识库中有一点关系的问题，就会简单回答。